In [1]:
import pandas as pd

In [2]:
word_counts = pd.read_csv('hw1_word_counts_05.txt', sep=" ", header=None)

In [3]:
word_counts.head(10)

,0,1
0,AARON,413
1,ABABA,199
2,ABACK,64
3,ABATE,69
4,ABBAS,290
5,ABBEY,213
6,ABBIE,42
7,ABBOT,20
8,ABDEL,337
9,ABDOU,16


In [4]:
word_counts.columns = ['word', 'count']

In [5]:
word_counts = word_counts.sort_values('count', ascending=False)

In [6]:
word_counts.head(15)

,word,count
5821,THREE,273077
5102,SEVEN,178842
1684,EIGHT,165764
6403,WOULD,159875
18,ABOUT,157448
5804,THEIR,145434
6320,WHICH,142146
73,AFTER,110102
1975,FIRST,109957
1947,FIFTY,106869


In [7]:
word_counts.tail(14)

,word,count
5093,SERNA,7
5872,TOCOR,7
3978,NIAID,7
1842,FABRI,7
4266,PAXON,7
2041,FOAMY,7
6443,YALOM,7
977,CCAIR,7
1107,CLEFT,7
3554,MAPCO,6


In [8]:
count_sum = sum(word_counts['count'])
count_sum

7664857

In [9]:
word_prob = word_counts['count'] / count_sum;
word_prob

5821    3.562715e-02
5102    2.333272e-02
1684    2.162650e-02
6403    2.085818e-02
18      2.054154e-02
            ...     
3554    7.827935e-07
895     7.827935e-07
4160    7.827935e-07
5985    7.827935e-07
712     7.827935e-07
Name: count, Length: 6535, dtype: float64

In [10]:
word_counts['prob'] = word_prob
word_counts

,word,count,prob
5821,THREE,273077,3.562715e-02
5102,SEVEN,178842,2.333272e-02
1684,EIGHT,165764,2.162650e-02
6403,WOULD,159875,2.085818e-02
18,ABOUT,157448,2.054154e-02
...,...,...,...
3554,MAPCO,6,7.827935e-07
895,CAIXA,6,7.827935e-07
4160,OTTIS,6,7.827935e-07
5985,TROUP,6,7.827935e-07


Assume `curr=['M','_','D','_','M']`, `block_list=['I']`:
$\text{best_next_guess}=argmax_{l}P(L_2=l\text{ or }L_4=l|L_1=M,L_2=D,L_5=M,L_2\notin\{D,I,M\},L_4\notin\{D,I,M\})$

In [35]:
def prob_l_at_any_i_given_w(curr: list, w: str, l: str) -> float:
    for i in range(0, len(curr)):
        if curr[i] == '_' and w[i] == l:
                return 1
    return 0

In [36]:
prob_l_at_any_i_given_w(['A','P','P','L','_'], 'APPLE', 'E')

1

In [37]:
def prob_e_given_w(curr: list, block_list: set, w: str) -> float:
    for i in range(0, len(curr)):
        if curr[i] == '_':
            if w[i] in block_list:
                return 0
        elif curr[i] != w[i]:
            return 0
    return 1

In [38]:
prob_e_given_w(['M','_','D','_','M'], {'M', 'D', 'I'}, 'MEDIM')

0

In [53]:
def best_next_guess(curr: list, block_list: list, word_counts: pd.DataFrame) -> [str, float]:
    # add chars in curr into block list
    block_list += [c for c in curr if c != '_']
    block_set = set(block_list)
    best_guess = None
    max_prob = -1
    for letter in ["A","B","C","D","E","F","G","H","I","J","K","L","M","N","O","P","Q","R","S","T","U","V","W","X","Y","Z"]:
        if letter in block_list:
            continue
        num = 0
        denom = 0
        for _,row in word_counts.iterrows():
            tmp = prob_e_given_w(curr, block_set, row['word'])
            num += prob_l_at_any_i_given_w(curr, row['word'], letter) * tmp * row['prob']
            denom += tmp * row['prob']
        p = num / denom
        if p > max_prob:
            max_prob = p
            best_guess = letter
    return [best_guess, max_prob]

In [54]:
best_next_guess(['_','_','_','_','_'], [], word_counts)

['E', 0.5394172389647954]

In [55]:
best_next_guess(['_','_','_','_','_'], ['E','A'], word_counts)

['O', 0.5340315651557564]

In [56]:
best_next_guess(['A','_','_','_','S'], [], word_counts)

['E', 0.771537162162162]

In [57]:
best_next_guess(['A','_','_','_','S'], ['I'], word_counts)

['E', 0.7127008416220354]

In [58]:
best_next_guess(['_','_','O','_','_'], ['A','E','M','N','T'], word_counts)

['R', 0.7453866259829712]

In [59]:
best_next_guess(['_','_','_','_','_'], ['E','O'], word_counts)

['I', 0.6365554141009615]

In [60]:
best_next_guess(['D','_','_','I','_'], [], word_counts)

['A', 0.820684523809524]

In [61]:
best_next_guess(['D','_','_','I','_'], ['A'], word_counts)

['E', 0.7520746887966807]

In [62]:
best_next_guess(['_','U','_','_','_'], ['A','E','I','O','S'], word_counts)

['Y', 0.6269651101630527]